In [1]:
import requests
import os
import pandas as pd
from dotenv import load_dotenv
import time

In [2]:
# Get API key
load_dotenv()

lego_key = os.getenv("LEGO_API_KEY")

In [3]:
# Define function to get data from any of the lego apis
def get_lego_data(endpoint, api_key, **kwargs):
    
    # Initialize variable to store paginated results
    data_list = []
    
    # Form request URL
    params = ''
    if len(kwargs) > 0:
        for key, value in kwargs.items():
            params += ("&" + str(key) + "=" + str(value))
    next_page = endpoint + "?key=" + api_key + params
    
    # Make requests
    while next_page is not None:
        response = requests.get(next_page)
        
        # Handle the case when requests are throttled
        if response == 429:
            print("Response 429")
            time.sleep(2)
            continue
            
        #Save the results
        response_json = response.json()
        data_list += response_json["results"]
        next_page = response_json["next"]
        
        #Print a status every 5 pages
        if len(data_list) % 10000 == 0:
            print("Downloaded {} records.".format(len(data_list)))
        time.sleep(1)

    # Convert json list to a pandas dataframe
    df = pd.json_normalize(data_list)

    # Confirm that the number of rows in the df match the expected count
    print("{} of {} total records received.".format(df.shape[0],response_json["count"]))
    
    return df

In [4]:
# Test function by getting data for all sets from 1980
sets_1980 = get_lego_data("https://rebrickable.com/api/v3/lego/sets/", lego_key, min_year=1980, max_year=1980)
sets_1980.head()

110 of 110 total records received.


,set_num,name,year,theme_id,num_parts,set_img_url,set_url,last_modified_dt
0,01-1,Chain Links,1980,453,25,https://cdn.rebrickable.com/media/sets/01-1/11...,https://rebrickable.com/sets/01-1/chain-links/,2023-03-23T05:54:08.228867Z
1,02-2,Digger Bucket Assembly,1980,453,3,https://cdn.rebrickable.com/media/sets/02-2/11...,https://rebrickable.com/sets/02-2/digger-bucke...,2023-03-23T08:36:27.960565Z
2,03-1,Doors and Fences,1980,443,10,https://cdn.rebrickable.com/media/sets/03-1/11...,https://rebrickable.com/sets/03-1/doors-and-fe...,2023-03-23T11:33:32.232557Z
3,04-1,Propellers and Wheels,1980,443,13,https://cdn.rebrickable.com/media/sets/04-1/11...,https://rebrickable.com/sets/04-1/propellers-a...,2023-03-23T11:37:32.225636Z
4,05-1,Hinges,1980,443,3,https://cdn.rebrickable.com/media/sets/05-1/11...,https://rebrickable.com/sets/05-1/hinges/,2023-03-23T08:27:14.660648Z


In [5]:
# Save Lego set data to csv
sets_1980.to_csv("data/sets_1980.csv", index=False)

In [ ]:
# Get data for all sets
sets = get_lego_data("https://rebrickable.com/api/v3/lego/sets/", lego_key)
sets.head()

Downloaded 10000 records.


In [ ]:
# Save Lego set data to csv
sets.to_csv("data/sets.csv", index=False)

In [ ]:
# Get data for all themes
themes = get_lego_data("https://rebrickable.com/api/v3/lego/themes/", lego_key)
themes.head()

In [ ]:
# Save Lego themes data to csv
themes.to_csv("data/themes.csv", index=False)